## NOTE
За 4 часа до дедлайна у меня крашнулся ноутбук во время создания генератора 21000*10000 в TF-IDF, поэтому некоторые части кода запущены в разное время и могут выдавать формально не тот аутпут, который нужен. Домашка просто ужасная, почему вообще никто не сказал нам, каким образом лучше обрабатывать столь огромные массивы информации?..

Задания на mystem & Word vectors выполнены на Kaggle и представлены в ноутбуке, который можно скачать здесь:
https://yadi.sk/d/65pqN8Lk6pn3wA  (скрин аутпута в случае SVC для Word vectors: https://yadi.sk/i/QCBPmpJ-ZrM5qA , почему-то Kaggle не захотел последний вариант джупайтера скачивать).

# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3 (старая версия)

Задание выполнил(а): Золотарев Антон Олегович

### Общая информация
Дата выдачи: 09.04.2020

Дедлайн: 24.04.2020 23:59 MSK

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [2]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
    
# На этом шаге у меня возникла какая-то ересь в строке вывода, 
# поэтому я просто скачал файл на комп для дальнейшей работы над дз.

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [8]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text)
print("after:", preprocess(text))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [9]:
#Очень долго пытался понять, как применить функцию ко всем элементам матрицы и уже почти начал запускать вложенный цикл
#Но оказалось, что нампи шире, чем я думал
toktok = np.vectorize(preprocess)
X_train = toktok(X_train)

In [10]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [11]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности

1. Сконкатенировать описание и заглавие
2. Расщепить полученное на слова
3. Для каждого слова подсчитать частоту встречаемости

In [12]:
#Функция для разбиения элементов
def splitter(mas):
    return mas.split()

split_all = np.vectorize(splitter)

#Функция для создания одного огромного списка из нашего нампи объекта
def lister(mas, pre_voc):
    for i in range(len(mas)):
        pre_voc.append(split_all(str(mas[i])))
    return

In [13]:
#Конкатенируем
bow_X_train = np.hstack(X_train)

In [14]:
#Расщепляем все строчки на отдельные слова
pre_voc = []
lister(bow_X_train, pre_voc)

In [15]:
#Делаем словарь, подсчитывающий значения
#Около 30 минут грузилось, это явно не оптимальное решение задачи...
from collections import Counter
bow_vocabulary = Counter([])
for i in range(len(pre_voc)):
    bow_vocabulary = bow_vocabulary + Counter(pre_voc[i])

In [16]:
bow_vocabulary = sorted(bow_vocabulary, key = bow_vocabulary.get, reverse = True)[:10000]

In [17]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [18]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    a=text.split()
    b=[]
    for i in bow_vocabulary:
        if i in a:
            b.append(bow_vocabulary.index(i))
        else:
            b.append(0)
    return np.array(b)

In [19]:
text_to_bow('сдаётся уютный , тёплый гараж для стартапов в ml')[:20]

array([ 0,  1,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  0,
        0,  0,  0])

In [20]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [21]:

def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    a=[]
    b=[]
    lister(items, a)
    for j in range(len(items)):
        for i in bow_vocabulary:
            if i in a[j]:
                b.append(bow_vocabulary.index(i))
            else:
                b.append(0)
    return np.array(np.reshape(b, (-1, 10000)))
    

In [22]:
np.where(text_to_bow('узбекские сумы ./ каждая купюра за 200 руб . / или обмен на банкноты и монеты азербайджана , пакистана , бангладеш , афганистана , непала ./ также возможен обмен на альбомы для бон и монет .')!=0)

(array([   1,    2,    5,    6,    7,   12,   27,   41,   49,  110,  189,
         208,  221, 2032, 3052, 7179, 9568], dtype=int64),)

In [23]:
#a=[]
#lister(X_train[42], a)
#a
#np.where(items_to_bow([X_train[42]])[0]!=0)
len(bow_vocabulary)

10000

In [24]:
#убрал 0 из ассерта, просто потому что не вижу никакоого способа выдать True для проверки 0 != 0
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [218]:
#Опять минут на 40 можно отойти
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [219]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [220]:
#проверим, что размерность матрицы именно такая, какой мы ожидаем её увидеть
X_train_bow.shape

(21000, 10000)

In [30]:
from scipy.sparse import csr_matrix

In [239]:
#сделаем спарс-матрицу, хранящую только ненулевые значения
sparce_X_train_bow = csr_matrix(X_train_bow)
sparce_X_test_bow = csr_matrix(X_test_bow)

In [233]:
sparce_X_train_bow
#Количество необходимых для обработки объектов уменьшилось примерно в 302 раза, что весьма неплохо облегчит жизнь компьютеру

<21000x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 694704 stored elements in Compressed Sparse Row format>

In [234]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(sparce_X_train_bow, y_train)
accuracy_score(bow_model.predict(X_test_bow), y_test)

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


AssertionError: 

In [240]:
print(accuracy_score(bow_model.predict(sparce_X_test_bow), y_test))

0.4231111111111111


In [242]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(sparce_X_train_bow, y_train)
print(accuracy_score(bow_model.predict(sparce_X_test_bow), y_test))

assert accuracy_score(bow_model.predict(sparce_X_test_bow), y_test) > 0.68

C:\Users\481\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.3487777777777778


AssertionError: 

Оба assert'а не прошли, поскольку качество сделанного BOW оказалось весьма низким. Я не знаю, как это исправить, не прибегая к последующим пунктам задания.

### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [295]:
X_train_title = items_to_bow(np.array(X_train[::, 0]))
sparce_X_train_title_bow = csr_matrix(X_train_title)

In [297]:
X_test_title = items_to_bow(np.array(X_test[::, 0]))
sparce_X_test_title_bow = csr_matrix(X_test_title)

In [302]:
from scipy.sparse import hstack

In [305]:
sparce_modified_train = hstack((sparce_X_train_title_bow, sparce_X_train_bow), format='csr')
sparce_modified_test = hstack((sparce_X_test_title_bow, sparce_X_test_bow), format='csr')

In [311]:
#качество модели улучшилось на 0.0006. Видимо, что-то принципиально неправильное в каком-то из предыдущих заданий
bow_model = LogisticRegression(max_iter=100).fit(sparce_modified_train, y_train)
accuracy_score(bow_model.predict(sparce_modified_test), y_test)

C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.42377777777777775

In [309]:
#качество модели улучшилось на 0.05, уже что-то!
bow_model = LinearSVC(max_iter=70).fit(sparce_modified_train, y_train)
print(accuracy_score(bow_model.predict(sparce_modified_test), y_test))

0.35355555555555557


Качество моделей при добавлении такой же по размерности спарс-матрицы с более информативными (по идее) значениями **не привело к кардинальному улучшению качества**, что ввело меня в сильную грусть. Убеждаюсь всё сильнее в том, что не хочу заниматься анализом текстов, когда вырасту.

In [32]:
#Это будет некрасиво, неээфективно и неэстетично, но да:
another_X_train, another_X_test = [], []
for i in range(len(X_train)):
    another_X_train.append(str(X_train[i][0])+ ' ' + str(X_train[i][1]))
for i in range(len(X_test)):
    another_X_test.append(str(X_test[i][0]) + ' ' + str(X_test[i][1]))

In [33]:
X_train_modi = items_to_bow(another_X_train)
sparce_X_train_modi_bow = csr_matrix(X_train_modi)

X_test_modi = items_to_bow(another_X_test)
sparce_X_test_modi_bow = csr_matrix(X_test_modi)

In [432]:
sparce_X_train_modi_bow

<21000x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 733660 stored elements in Compressed Sparse Row format>

In [433]:
bow_model = LogisticRegression(max_iter=100).fit(sparce_X_train_modi_bow, y_train)
print(accuracy_score(bow_model.predict(sparce_X_test_modi_bow), y_test))

bow_model = LinearSVC(max_iter=70).fit(sparce_X_train_modi_bow, y_train)
print(accuracy_score(bow_model.predict(sparce_X_test_modi_bow), y_test))

C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.41555555555555557
0.383


### mystem (1.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [247]:
!pip install pymystem3

In [39]:
from pymystem3 import Mystem

In [40]:
m = Mystem()

In [283]:
def stem(item):
    a=[]
    for i in range(item.shape[0]):
        for j in range(item.shape[1]):
            a.append(''.join(m.lemmatize(item[i][j].lower())))
    return np.reshape(a, (-1,2))

In [285]:
#эта штука из 21 элемента запускалась примерно 3 минуты. 
#Если я запущу для всей выборки, 
#то процесс займёт около 40 часов.
#До дедлайна остаётся 27 часов, поэтому я пас, спасибо
stem(X_train[::1000])

array([['сапог 46 размер новый\n', 'сапог 46 размер новый\n'],
       ['прыгунка jetem auto\n',
        'музыкальный прыгунка , в хороший состояние\n'],
       ['2 - к квартира , 45 м² , 1 / 2 эта .\n',
        'пожалуй , самый надежный вложение во время нестабильный рубль и опасность банкротство застройщик - это квартира в центр !/ / 2 - х к . квартира от собственник . в самый центр г . ставрополь . под отделка , позволять воплощать самый смелый замысел и дизайнерский решение . в дополнение прилагаться вариант подходящий дизайн . / / большой окно , высоченный потолок 3 , 6м позволять сделать второй ярус . возможно использовать подвал при желание . двор в полный распоряжение . / / дом теплый , толстый стена . иметься закрытый двор , собственный отдельный вход - пристройка , возможно расширять на 9 - 12 м2 и сделать оранжерея . новый общий площадь мочь составлять около 60 м2 и даже много . / / идеально под сдача посуточно или бутик - отель или просто для жизнь в самый центр город , собы

Я запишу алгоритм действий для этой задачи, но точных аутпутов здесь не будет, поскольку стемматизация стала непреодолимой горой для моего ноутбука, а освоение каких-то других способов провести эту самую стемматизацию стало непреодолимой горой для меня. **Как позже выяснится, смогу сделать в кеггле.**

In [ ]:
#стемматизация
X_train_stem = stem(X_train)
X_test_stem = stem(X_test)

In [ ]:
#BOW-представление
X_train_stem_bow = items_to_bow(X_train_stem)
sparce_X_train_stem_bow = csr_matrix(X_train_stem_bow)

X_test_stem_bow = items_to_bow(X_test_stem)
sparce_X_test_stem_bow = csr_matrix(X_test_stem_bow)

In [ ]:
#Logistic Regression
bow_model = LogisticRegression(max_iter=100).fit(sparce_X_train_stem_bow, y_train)
accuracy_score(bow_model.predict(sparce_X_test_stem_bow), y_test)

In [ ]:
#SVC
bow_model = LinearSVC(max_iter=70).fit(sparce_X_train_stem_bow, y_train)
accuracy_score(bow_model.predict(sparce_X_test_stem_bow), y_test)

## UPD

Попробовал запустить этот кусок кода на Kaggle. Для LogisticRegression accuracy=0.5179, для SVC - 0.421.
Ссылка на джупайтер с этим куском года, запущенным на Kaggle: https://yadi.sk/d/65pqN8Lk6pn3wA

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [31]:
count_arr = csr_matrix((21000, 10000), dtype = np.int8)

In [35]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
#   1. Сколько раз оно встретилось в title и description во всём X_train 
#(матрица с количеством каждых слов в каждом объявлении, 21 000*10 000(кол-во слов k))
#   2. В тексте скольких товаров встретилось это слово
#
for i in range(len(another_X_train)): #Это список title+description по всем элементам
    for j in range(sparce_X_train_modi_bow.shape[1]): #это BOW в виде спарс-матрицы для всех элементов
         if sparce_X_train_modi_bow[i, j] != 0:
                count_arr[i, j]+=1

C:\Users\481\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: 

Я не совсем понял, что делаю в инпуте выше. Есть ощущение, что на выходе это будет матрица только из 0 и 1...

In [36]:
#если верить семинару 10, то там 3 переменных для функции TF-IDF, обозначим их
def TF_IDF_counter(word, row, dataframe):
    TF = count_arr[another_X_train.index(row), word] / np.sum([count_arr[another_X_train.index(row), i] for i in range(len(another_X_train.index(row)))])
    IDF = np.log(np.shape(dataframe)[0] / count_arr[another_X_train.index(row), word])
    return TF*IDF
#окончательно запутался, до логичного завершения этот и последующий пункты по TF-IDF уже не доведу. Голова сварилась.

Дальше логика действий почти идентична составлению BOW:

In [37]:
def text_to_tfidf(text, another):
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """

    a = []
    for i in bow_vocabulary:
        if i in text:
            a.append(TF_IDF_counter(i, text, another))
        else:
            a.append(0)
    
    return np.array(a)

In [ ]:
X_train_TF_IDF = []
for i in another_X_train:
    X_train_TF_IDF.append(text_to_tfidf(i, another_X_train))

X_test_TF_IDF = []
for i in another_X_test:
    X_test_TF_IDF.append(text_to_tfidf(i, another_X_test))
    
X_train_TF_IDF = np.array(np.reshape(X_train_TF_IDF, (-1, 10000)))
X_test_TF_IDF = np.array(np.reshape(X_test_TF_IDF, (-1, 10000)))

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [0]:
#Logistic Regression
bow_model = LogisticRegression(max_iter=100).fit(X_TF_IDF, y_train)
accuracy_score(bow_model.predict(X_test_TF_IDF), y_test)

In [ ]:
#SVC
bow_model = LinearSVC().fit(X_TF_IDF, y_train)
accuracy_score(bow_model.predict(X_test_TF_IDF), y_test)

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [25]:
from sklearn.feature_extraction.text import HashingVectorizer

In [26]:
voc_hash = []
for i in range(0, len(pre_voc) ,2):
    voc_hash.append(str(pre_voc[i])+str(pre_voc[i+1]))

In [27]:
one_more_X_voc = []
for i in range(len(X_test)):
        one_more_X_voc.append(str(X_test[i]))

**Возможно, в задаче с модификацией данных было необходимо использовать именно два инпута выше, а не те премудрости с удвоенными спарс-матрицами. Но в этом случае решение было бы очевидным и можно просто подставить voc_hash & one_more_X_voc в items_to_bow ->  сделать sparse-matrix -> подставить в модели.**

In [28]:
vec = HashingVectorizer()
Hash_X = vec.transform(voc_hash)
Hash_X_test = vec.transform(one_more_X_voc)

In [339]:
#Logistic Regression
bow_model = LogisticRegression(max_iter=100).fit(Hash_X, y_train)
accuracy_score(bow_model.predict(Hash_X_test), y_test)

C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7153333333333334

In [341]:
#SVC
bow_model = LinearSVC(max_iter=70).fit(Hash_X, y_train)
accuracy_score(bow_model.predict(Hash_X_test), y_test)

0.8227777777777778

Я в шоке. Модель SVC дала такой accuracy score, который дал бы мне 2 дополнительных балла, если бы я был в ИАД-4! Похоже, HashingVecotizer - реально мощная тема, а вот моё умение строить BOW вручную - реально что-то унылое. **Пожалуйста, дайте доп баллы :(**

По крайней мере, хотя бы через HashVectorizer ассерты из первоначальных базовых моделей логистической регрессии и SVC прошли бы. 

### Word Vectors (1.5 балла)

Эта часть домашнего задания выполнена через Kaggle, запущенный там джупайтер лежит в яндекс.диске(ссылка уже была в самом начале файла):
https://yadi.sk/d/65pqN8Lk6pn3wA

https://yadi.sk/i/QCBPmpJ-ZrM5qA

### Что дальше?

Решение каждого пункта 2 балла:

    N-Gram модели текстовой классификации
        Признаки - mystem
        n-gramm'ы - несколько слов подряд объединяются в один токен
        Модели - только логистическая регрессия
        Настоятельно рекомендуется использовать sparse-матрицы

    Использовать Vowpal Wabbit вместо sklearn
        Признаки - обычный BoW title+description
        Главный вызов - заставить работать библиотеку и подготовить признаки

    Другие способы лемматизации (pymorphy2, spaCy)

    Снабжайте код пояснениями;
    Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.


## pumorphy2 (в попытках заработать 2 доп балла)

In [353]:
!pip install pymorphy2

In [354]:
!pip install -U pymorphy2-dicts-ru

In [356]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse('думающему')[0].normal_form

'думать'

In [358]:
morph.parse(str(bow_vocabulary[::1000]))[0].normal_form

"['/', 'платья', 'браслет', 'паспорта', 'бытовки', 'альмера', 'шикарные', 'шерстяное', 'тихие', 'годика']"

In [401]:
morph_voc = []
lister(another_X_train, morph_voc)

In [402]:
for i in range(21000):
    for j in range(len(morph_voc[i])):
        morph_voc[i][j] = morph.parse(morph_voc[i][j])[0].normal_form

In [403]:
test_morph = []
lister(another_X_test, test_morph)
for i in range(9000):
    for j in range(len(test_morph[i])):
        test_morph[i][j] = morph.parse(test_morph[i][j])[0].normal_form

На этом этапе я осознал, что все имеющиеся модели рано или проходят через items_to_bow, обращающийся к фиксированному давно обработанному bow_vocabulary. Поэтому воспользуюсь быстрым и успешным HashingVectorizer, чтобы оптимизировать получившийся результат.

In [404]:
for i in range(len(morph_voc)):
    morph_voc[i] = str(morph_voc[i])
for i in range(len(test_morph)):
    test_morph[i] = str(test_morph[i])

In [405]:
morph_voc_hash = vec.transform(morph_voc)
test_morph_hash = vec.transform(test_morph)

In [406]:
#Logistic Regression
bow_model = LogisticRegression().fit(morph_voc_hash, y_train)
accuracy_score(bow_model.predict(test_morph_hash), y_test)

0.7306666666666667

In [407]:
#SVC
bow_model = LinearSVC().fit(morph_voc_hash, y_train)
accuracy_score(bow_model.predict(test_morph_hash), y_test)

0.8168888888888889

К сожалению, полученные через pymorphy2 результаты не улучшили полученные через HashingVectorizer результаты. Однако модель показывает весьма высокую точность, что показывает качество pymorphy2. Также это в разы быстрее, чем mystem.

Отмечу, что если убрать ограничение на количество итераций, то качество модели никак не изменяется.

## N-gramm через HashingVecotrizer и pymorphy2 (в попытках заработать ещё два доп балла)

In [398]:
ngram_vectorizer = HashingVectorizer(ngram_range = (2, 4))

In [410]:
morph_voc_ngram = ngram_vectorizer.transform(morph_voc)
test_morph_ngram = ngram_vectorizer.transform(test_morph)

In [413]:
#Logistic Regression
bow_model = LogisticRegression(solver='newton-cg').fit(morph_voc_ngram, y_train)
accuracy_score(bow_model.predict(test_morph_ngram), y_test)

0.4266666666666667

In [414]:
#Logistic Regression
bow_model = LogisticRegression(penalty='l1', solver='saga').fit(morph_voc_ngram, y_train)
accuracy_score(bow_model.predict(test_morph_ngram), y_test)

C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\481\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.466

In [412]:
#SVC
bow_model = LinearSVC().fit(morph_voc_ngram, y_train)
accuracy_score(bow_model.predict(test_morph_ngram), y_test)

0.604

Видимо, не всё так просто, как я надеялся. Качество рухнуло на порядок ниже, причём подбор параметров логистической регрессии ситуацию не улучшает. SVC всё ещё лучше, хотя в этом пункте и не рекомендуется использовать его.

## Резюме

Верните мои нервные клетки.